In [1]:
from ppq import *
from ppq.api import *
import torch
import cfg
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "1,2"


      ____  ____  __   ____                    __              __
     / __ \/ __ \/ /  / __ \__  ______ _____  / /_____  ____  / /
    / /_/ / /_/ / /  / / / / / / / __ `/ __ \/ __/ __ \/ __ \/ /
   / ____/ ____/ /__/ /_/ / /_/ / /_/ / / / / /_/ /_/ / /_/ / /
  /_/   /_/   /_____\___\_\__,_/\__,_/_/ /_/\__/\____/\____/_/




In [2]:
from turtle import width
import onnx
import onnx.checker
import onnx.utils
from onnx.tools import update_model_dims

model = onnx.load('/home/geng/tinyml/ppq/benchmark/detection/FP32_model/Retinanet-FP32.onnx')
# 此处可以理解为获得了一个维度 “引用”，通过该 “引用“可以修改其对应的维度  
                                                                                     
height = model.graph.input[0].type.tensor_type.shape.dim[2]
# 将该维度赋值为字符串，其维度不再为和dummy_input绑定的值
height.dim_param = 'height'

width = model.graph.input[0].type.tensor_type.shape.dim[3]
# 将该维度赋值为字符串，其维度不再为和dummy_input绑定的值
width.dim_param = 'width'

onnx.save(model, '/home/geng/tinyml/ppq/benchmark/detection/FP32_model/Retinanet-dynamic-FP32.onnx')

In [2]:
# 消除冗余算子
# import onnx
# from onnxsim import simplify

# onnx_file = "/home/geng/tinyml/ppq/benchmark/detection/FP32_model/Retinanet-FP32-end-11.onnx"
# onnx_model = onnx.load(onnx_file) 
# model_simp, check = simplify(onnx_model)   #对onnx模型进行简化，消除冗余算子        
# assert check, "Simplified ONNX model could not be validated"
# onnx.save(model_simp, onnx_file)

In [3]:
# 升级模型
import onnx
from onnx import version_converter, helper

# # Preprocessing: load the model to be converted.
# model_path = "/home/geng/tinyml/ppq/benchmark/detection/FP32_model/Retinanet-FP32.onnx"
# original_model = onnx.load(model_path)

# # A full list of supported adapters can be found here:
# # https://github.com/onnx/onnx/blob/main/onnx/version_converter.py#L21
# # Apply the version conversion on the original model
# converted_model = version_converter.convert_version(original_model, 11)

# onnx.save(converted_model, "/home/geng/tinyml/ppq/benchmark/detection/FP32_model/Retinanet-FP32.onnx")

In [2]:
model_path = "/home/geng/tinyml/ppq/benchmark/detection/FP32_model/Retinanet-FP32.onnx"
# model_path = "/home/geng/tinyml/ppq/benchmark/detection/FP32_model/end2end-12.onnx"

from dataset import build_dataset
import torch
import cfg

model_name = "Retinanet"
input_size = cfg.MODELS[model_name]["INPUT_SHAPE"]
# 获取校准数据集
_,calib_dataloader =  build_dataset(ann_file=cfg.ANN_PATH,data_root=cfg.DATA_ROOT,
        input_size=input_size,batch_size=cfg.CALIBRATION_BATCH_SIZE)

# calib_dataloader = [torch.rand(size=input_size) for _ in range(512)]


config = cfg.PLATFORM_CONFIGS["TRT"]
config["QuanSetting"].dispatcher = "conservative"

loading annotations into memory...
Done (t=0.82s)
creating index...
index created!


In [3]:
ppq_quant_ir = quantize_onnx_model(
    onnx_import_file=model_path, calib_dataloader=calib_dataloader, calib_steps=8 // cfg.CALIBRATION_BATCH_SIZE, 
    setting=config["QuanSetting"],input_shape=input_size, collate_fn=lambda x: x["img"][0].to(cfg.DEVICE), 
    platform=config["QuantPlatform"], do_quantize=True)

[18:29:18] PPQ Quantization Config Refine Pass Running ... Finished.
[18:29:18] PPQ Quantization Fusion Pass Running ...        Finished.
[18:29:19] PPQ Quantize Simplify Pass Running ...          Finished.
[18:29:19] PPQ Parameter Quantization Pass Running ...     

Calibration Progress(Phase 1):   0%|          | 0/512 [00:00<?, ?it/s]

Finished.
[18:29:19] PPQ Runtime Calibration Pass Running ...        

Calibration Progress(Phase 1):   0%|          | 1/512 [00:07<1:04:11,  7.54s/it]


KeyboardInterrupt: 

In [31]:
from inference import ppq_inference
dataloader = [next(iter(calib_dataloader))]
outputs = ppq_inference(dataloader=dataloader,ppq_ir = ppq_quant_ir,device = "cuda")

  0%|          | 0/1 [00:00<?, ?it/s]


RuntimeError: Error happens when dealing with operation Conv_37(TargetPlatform.TRT_INT8) - inputs:['input', 'onnx::Conv_1245', 'onnx::Conv_1246'], outputs:['input.3']

In [28]:
def post_process(model_type,outputs,class_num):
    """
    将任意模型的输出转换为常规输出
    [N,result]  N为样本数
    result: [C,[M,5]] 其中C为类别,M为bbox数目，5为左上角、右下角坐标与置信分数。
    并且根据尺度进行复原
    """
    if model_type == "Retinanet":
        print("deal with outputs in Retinanet type")
        results = []
        for output in outputs:
            img_scale = output["scale_factor"] 
            labels,bboxs = output["output"]
            result = [[] for _ in range(class_num)]
            if len(labels.shape) > 2:
                # labels 和 bbox的输出顺序发生错换，需要调整
                bboxs,labels = labels,bboxs
                
            bboxs,labels = bboxs[0],labels[0]
            for i,label in enumerate(labels):
                bboxs[i][:4] = bboxs[i][:4] / img_scale  # 进行bbox的尺寸复原
                result[label].append(bboxs[i])
            results.append(result)
        return results
    elif model_type == "fasterrcnn":
        # TODO
        pass 

In [29]:
results = post_process(model_name,outputs,80)

deal with outputs in Retinanet type


In [14]:
model_name = "Retinanet"
export_ppq_graph(   graph = ppq_quant_ir,
                    platform=TargetPlatform.ONNXRUNTIME,
                    graph_save_to=f'{os.path.join(config["OutputPath"], model_name)}-ORT-END-INT8.onnx')

In [7]:
x = next(iter(calib_dataloader))

In [8]:
x["img"][0].shape

torch.Size([2, 3, 800, 1216])

In [4]:
calib_input_size

(512, 3, 800, 1216)

In [ ]:
"""
op Less need opset (12,16)
op TopK need opset (1,12)
Upsample not supported
"""

In [8]:
import cv2
from PIL import Image
import numpy as np
image_path = "/home/geng/fiftyone/coco-2017/validation/data/000000061418.jpg"
image = Image.open(image_path)

ratio = 800.0 / min(image.size[0], image.size[1])
image = image.resize((int(ratio * image.size[0]), int(ratio * image.size[1])),  Image.Resampling.BILINEAR)
image = cv2.cvtColor(np.array(image) , cv2.COLOR_GRAY2RGB)

In [9]:
np.array(image).shape

(800, 1117, 3)

In [11]:
image[:,:,0]

array([[ 96, 109, 130, ..., 208, 213, 217],
       [ 93, 106, 126, ..., 211, 213, 215],
       [ 89, 101, 119, ..., 215, 213, 212],
       ...,
       [116, 117, 119, ...,  94,  94,  93],
       [107, 108, 112, ..., 103, 106, 108],
       [101, 103, 107, ..., 109, 114, 117]], dtype=uint8)

In [5]:
import numpy as np
a = np.random.rand(10,15)
b = np.random.rand(5,5)
a = (a > 0.5).astype("int")
b = np.asfortranarray(a)

In [14]:
a[10:10,3:5] = b[0:0,1:2]

In [1]:

def preprocess(image):
    channel_num = image.layers  #通道数

    # Resize
    ratio = 800.0 / min(image.size[0], image.size[1])
    image = image.resize((int(ratio * image.size[0]), int(ratio * image.size[1])),  Image.Resampling.BILINEAR)
    
    if channel_num == 1:  #灰度图像转三通道
        # print("trans")
        image = cv2.cvtColor(np.array(image) , cv2.COLOR_GRAY2RGB)
   
    # Convert to BGR
    image = np.array(image)[:, :, [2, 1, 0]].astype('float32')

    # HWC -> CHW
    image = np.transpose(image, [2, 0, 1])

    # Normalize
    mean_vec = np.array([102.9801, 115.9465, 122.7717])
    for i in range(image.shape[0]):
        image[i, :, :] = image[i, :, :] - mean_vec[i]

    # Pad to be divisible of 32
    import math
    padded_h = int(math.ceil(image.shape[1] / 32) * 32)
    padded_w = int(math.ceil(image.shape[2] / 32) * 32)

    padded_image = np.zeros((3, padded_h, padded_w), dtype=np.float32)
    padded_image[:, :image.shape[1], :image.shape[2]] = image
    image = padded_image

    return image

In [2]:
from inference import TrtInferenceModel
import ctypes
import numpy as np

ctypes.CDLL("/home/geng/tinyml/ppq/benchmark/detection/lib/libmmdeploy_tensorrt_ops.so")

trt_model_path = '/home/geng/tinyml/ppq/benchmark/detection/dynamic_input_test/FasterRCNN-NMS-FP32.engine'
trtmodel = TrtInferenceModel(trt_model_path)

with open("/home/geng/tinyml/ppq/benchmark/detection/images_list.txt","r") as f:
   img_path_list =  f.readlines()
img_path_list  = [x.rstrip() for x in img_path_list]
img_path = img_path_list[0]

from PIL import Image
engine_path ='/home/geng/tinyml/ppq/benchmark/detection/dynamic_input_test/FasterRCNN-NMS-FP32.engine'
img = Image.open(img_path)
input_tensor = preprocess(img)[None,:,:,:]
print(input_tensor.shape)


      ____  ____  __   ____                    __              __
     / __ \/ __ \/ /  / __ \__  ______ _____  / /_____  ____  / /
    / /_/ / /_/ / /  / / / / / / / __ `/ __ \/ __/ __ \/ __ \/ /
   / ____/ ____/ /__/ /_/ / /_/ / /_/ / / / / /_/ /_/ / /_/ / /
  /_/   /_/   /_____\___\_\__,_/\__,_/_/ /_/\__/\____/\____/_/


[09/20/2022-18:10:23] [TRT] [I] [MemUsageChange] Init CUDA: CPU +310, GPU +0, now: CPU 395, GPU 1761 (MiB)
[09/20/2022-18:10:23] [TRT] [I] Loaded engine size: 196 MiB


/home/geng/anaconda3/envs/ppq/lib/python3.8/site-packages/tensorrt/__init__.py:75: DeprecationWarning: Context managers for TensorRT types are deprecated. Memory will be freed automatically when the reference count reaches 0.
  warnings.warn(


[09/20/2022-18:10:24] [TRT] [I] [MemUsageChange] Init cuBLAS/cuBLASLt: CPU +496, GPU +214, now: CPU 1111, GPU 2171 (MiB)
[09/20/2022-18:10:24] [TRT] [I] [MemUsageChange] Init cuDNN: CPU +392, GPU +200, now: CPU 1503, GPU 2371 (MiB)
[09/20/2022-18:10:24] [TRT] [W] TensorRT was linked against cuDNN 8.4.1 but loaded cuDNN 8.2.2
[09/20/2022-18:10:24] [TRT] [I] [MemUsageChange] TensorRT-managed allocation in engine deserialization: CPU +0, GPU +211, now: CPU 0, GPU 211 (MiB)
[09/20/2022-18:10:25] [TRT] [I] [MemUsageChange] Init cuBLAS/cuBLASLt: CPU +0, GPU +8, now: CPU 1306, GPU 2379 (MiB)
[09/20/2022-18:10:25] [TRT] [I] [MemUsageChange] Init cuDNN: CPU +0, GPU +8, now: CPU 1306, GPU 2387 (MiB)
[09/20/2022-18:10:25] [TRT] [W] TensorRT was linked against cuDNN 8.4.1 but loaded cuDNN 8.2.2
[09/20/2022-18:10:25] [TRT] [I] [MemUsageChange] TensorRT-managed allocation in IExecutionContext creation: CPU +0, GPU +579, now: CPU 0, GPU 790 (MiB)
[09/20/2022-18:10:25] [TRT] [W] The getMaxBatchSize() 

/home/geng/tinyml/ppq/benchmark/detection/inference/trt_infer.py:134: DeprecationWarning: Use network created with NetworkDefinitionCreationFlag::EXPLICIT_BATCH flag instead.
  size = trt.volume(engine.get_binding_shape(binding)) * engine.max_batch_size


In [3]:
outputs = trtmodel(input_tensor.astype(np.float32))

LogicError: cuMemcpyHtoDAsync failed: invalid argument

In [ ]:
import onnx
print(f"正在获取预训练模型：{name}")
dump_torch_to_onnx(model=model, onnx_export_file=f'{os.path.join(cfg.FP32_BASE_PATH, name)}-FP32.onnx',
    input_shape=cfg.INPUT_SHAPE,input_dtype=torch.float,device="cpu")

onnx_model = onnx.load(f'{os.path.join(cfg.FP32_BASE_PATH, name)}-FP32.onnx') 
model_simp, check = simplify(onnx_model)   #对onnx模型进行简化，消除冗余算子        
assert check, "Simplified ONNX model could not be validated"
onnx.save(model_simp, f'{os.path.join(cfg.FP32_BASE_PATH, name)}-FP32.onnx')